In [1]:
import os

os.environ['DJANGO_SETTINGS_MODULE'] = 'cashup_backend.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = "true"

import django
django.setup()

In [2]:
from data.models import HourData

In [3]:
import requests
from datetime import timedelta, datetime

In [4]:
up_dn_list = []
last_data = []
dn_list = []
up_list = []
new_dn_list = []
new_up_list = []
other_up_dn_list = []

continue_up_down = "D0"
prev_up_down = "S"
work_1_0_1 = "S"
work_1_5_1 = "S"
work_1_0_2 = "S"
work_1_5_2 = "S"
last_time_1_0_1 = datetime.now()
last_time_1_5_1 = datetime.now()
last_time_1_0_2 = datetime.now()
last_time_1_5_2 = datetime.now()

last_dn, last_up = "", ""

max_price, min_price = 0, 0
work_up_high, work_dn_low = 1000000, 0
volume = 0
prev_low = 0
prev_high = 100000
start = 0
last_up = 0
last_dn = 0
volume_up_dn = ""
hour_up_down = ""
prev_signal_up_price = 0
prev_signal_down_price = 0

In [5]:

while True:
    URL = "https://www.bitmex.com/api/v1/trade/bucketed?symbol=XBT&binSize=1h&partial=false&start={}&count=1000&reverse=false".format(start)
    req = requests.get(URL).json()
    for data in req:
        volume_up_dn = ""
        signal = ""
        signal_price = 0
        hour_up_down = ""
        now_work_1_0_1 = ""
        now_work_1_5_1 = ""
        now_work_1_0_2 = ""
        now_work_1_5_2 = ""
        datetime = datetime.strptime(data['timestamp'].replace("T", " ")[0:19], "%Y-%m-%d %H:%M:%S") + timedelta(hours=8)
        open_price = data['open'] if data['open'] != None else 0
        high_price = data['high'] if data['high'] != None else 0
        low_price = data['low'] if data['low'] != None else 0
        close_price = data['close'] if data['close'] != None else 0
        avg_price = (high_price + low_price) / 2
        
        if volume != 0:
            volume_rate = data['volume'] / volume
        else:
            volume_rate = 0
            
        volume = data['volume']

        flag = False

        if open_price > close_price:
            UD = "D"

        elif open_price < close_price:
            UD = "U"

        else:
            UD = "S"

        if len(last_data) > 3:
            last_data.pop(0)
            volume_bigger_count = 0
            volume_smaller_count = 0

            for i in last_data:
                if i[2] < volume:
                    volume_bigger_count += 1

                if i[2] > volume:
                    volume_smaller_count += 1

            if volume_bigger_count == 3:
                volume_up_dn = "UP"

            if volume_smaller_count == 3:
                volume_up_dn = "DN"

        last_data.append([low_price, high_price, volume])
        
        if continue_up_down[0] == "U":
            if UD == "U":
                continue_up_down = "U" + str(int(continue_up_down[1:]) + 1)
            elif UD == "D":
                continue_up_down = "D1"
        else:
            if UD == "U":
                continue_up_down = "U1"
            elif UD == "D":
                continue_up_down = "D" + str(int(continue_up_down[1:]) + 1)
        
        if len(up_dn_list) > 3:
            up_dn_list.pop(0)
        up_dn_list.append(UD)
        
        if up_dn_list.count("U") > 2:
            if high_price >= last_up:
                hour_up_down = "UP(U)"
            else:
                hour_up_down = "UP(D)"
            last_up = high_price
        
        if up_dn_list.count("D") > 2:
            if low_price >= last_dn:
                hour_up_down = "DN(U)"
            else:
                hour_up_down = "DN(D)"
            last_dn = low_price
            
        if UD == "U":
            if other_up_dn_list.count("D") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                for idx, element in enumerate(other_up_dn_list):
                    if element == "D":
                        count += 1
                    elif element == "U":
                        if count >= 3:
                            break
                    max_price_list.append(dn_list[idx])
                    min_price_list.append(up_list[idx])

                if prev_up_down == "D" and max(max_price_list) == max_price_list[len(max_price_list) - 1]:
                    if prev_signal_down_price != 0:
                        if prev_signal_down_price < min(min_price_list):
                            signal = "fD(U)"
                        elif prev_signal_down_price == min(min_price_list):
                            signal = "fD(0)"
                        else:
                            signal = "fD(D)"
                    prev_signal_down_price = min(min_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()

        if UD == "D":
            if other_up_dn_list.count("U") > 2:
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
                count = 0
                max_price_list = []
                min_price_list = []
                is_D = False
                for idx, element in enumerate(other_up_dn_list):
                    if element == "U":
                        count += 1
                    elif element == "D":
                        if count >= 3:
                            break
                    max_price_list.append(dn_list[idx])
                    min_price_list.append(up_list[idx])

                if prev_up_down == "U" and min(min_price_list) == min_price_list[len(min_price_list) - 1]:
                    print(datetime)
                    print(other_up_dn_list)
                    print(max_price_list, min_price_list)
                    if prev_signal_down_price != 0:
                        if prev_signal_up_price < max(max_price_list):
                            signal = "fU(U)"
                        elif prev_signal_up_price == max(max_price_list):
                            signal = "fU(0)"
                        else:
                            signal = "fU(D)"
                    prev_signal_up_price = max(max_price_list)
                other_up_dn_list.reverse()
                up_list.reverse()
                dn_list.reverse()
        
        if len(dn_list) > 5:
            dn_list.pop(0)
        dn_list.append(high_price)
            
        if len(up_list) > 5:
            up_list.pop(0)
        up_list.append(low_price)
        
        if len(other_up_dn_list) > 5:
            other_up_dn_list.pop(0)
        other_up_dn_list.append(UD)
        
        if volume_up_dn == "UP":
            if (max_price < high_price) and (volume_rate > 1):
                if work_1_0_1[0] == "U":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0_1 = True
                    else:
                        work_1_0_1 = "U" + str(int(work_1_0_1[1:]) + 1)
                else:
                    work_1_0_1 = "U1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1):
                if work_1_0_1[0] == "D":
                    if datetime == last_time_1_0_1 + timedelta(hours=1):
                        continue_flag_1_0 = True
                    else:
                        work_1_0_1 = "D" + str(int(work_1_0_1[1:]) + 1)
                
                else:
                    work_1_0_1 = "D1"
                
                now_work_1_0_1 = work_1_0_1
                last_time_1_0_1 = datetime
                
            if (max_price < high_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "U":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        continue_flag_1_5_1 = True
                    else:
                        work_1_5_1 = "U" + str(int(work_1_5_1[1:]) + 1)
                else:
                    work_1_5_1 = "U1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime
            
            if (min_price > low_price) and (volume_rate > 1.5):
                if work_1_5_1[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_1 = "D" + str(int(work_1_5_1[1:]) + 1)
                
                else:
                    work_1_5_1 = "D1"
                
                now_work_1_5_1 = work_1_5_1
                last_time_1_5_1 = datetime

            if (max_price < high_price and min_price < low_price) and (volume_rate > 1):
                if work_1_0_2[0] == "U":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "U" + str(int(work_1_0_2[1:]) + 1)
                else:
                    work_1_0_2 = "U1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1):
                if work_1_0_2[0] == "D":
                    if datetime == last_time_1_0_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_0_2 = "D" + str(int(work_1_0_2[1:]) + 1)
                
                else:
                    work_1_0_2 = "D1"
                
                now_work_1_0_2 = work_1_0_2
                last_time_1_0_2 = datetime     
                
                
            if (max_price < high_price and min_price < low_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "U":
                    if datetime == last_time_1_5_2 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "U" + str(int(work_1_5_2[1:]) + 1)
                else:
                    work_1_5_2 = "U1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime
            
            if (min_price > low_price and max_price > high_price) and (volume_rate > 1.5):
                if work_1_5_2[0] == "D":
                    if datetime == last_time_1_5_1 + timedelta(hours=1):
                        pass
                    else:
                        work_1_5_2 = "D" + str(int(work_1_5_2[1:]) + 1)
                
                else:
                    work_1_5_2 = "D1"
                
                now_work_1_5_2 = work_1_5_2
                last_time_1_5_2 = datetime

            max_price, min_price = high_price, low_price
            
        if now_work_1_0_1 != "":
            now_work_1_0_1 = "w" + now_work_1_0_1
        
        if now_work_1_5_1 != "":
            now_work_1_5_1 = "w" + now_work_1_5_1
            
        if now_work_1_0_2 != "":
            now_work_1_0_2 = "w" + now_work_1_0_2
            
        if now_work_1_5_2 != "":
            now_work_1_5_2 = "w" + now_work_1_5_2
        
        print(hour_up_down)
        element = HourData()
        element.datetime = datetime
        element.open_price = open_price
        element.min_price = low_price
        element.max_price = high_price
        element.close_price = close_price
        element.avg_price = avg_price
        element.volume = volume
        element.volume_rate = volume_rate
        element.volume_up_dn = volume_up_dn
        element.up_down = UD
        element.continue_up_down = continue_up_down
        element.hour_up_down = hour_up_down
        element.signal = signal
        element.signal_price = signal_price
        element.work_two_1_0 = now_work_1_0_1
        element.work_two_1_5 = now_work_1_5_1
        element.work_one_1_0 = now_work_1_0_2
        element.work_one_1_5 = now_work_1_5_2
        element.save()
        
        prev_up_down = UD
        
    start += 1000




DN(U)




DN(D)
DN(D)
DN(U)

UP(U)
2015-09-26 10:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[235.29, 235.36, 235.2] [235.29, 235.19, 234.9]
UP(U)








DN(D)
DN(D)


UP(D)
2015-09-27 00:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[235, 234.35, 234.87, 233.85] [235, 234.34, 234.1, 232.91]

DN(U)


UP(D)
UP(D)
2015-09-27 06:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[234.67, 234.73, 234, 233.8] [234.32, 234.05, 233.69, 233.68]
UP(D)
UP(D)












DN(D)
DN(U)
DN(D)





UP(D)


DN(U)
DN(D)
DN(D)
DN(U)

UP(U)
UP(D)
UP(D)
2015-09-28 15:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[237.8, 237.82, 238.23, 234.58, 233.93] [237.37, 237.34, 234.45, 233.6, 233.39]
UP(D)
UP(U)
2015-09-28 17:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[238.7, 237.24, 237.8, 237.82, 238.23, 234.58] [237.3, 237.03, 237.37, 237.34, 234.45, 233.6]


UP(U)
2015-09-28 21:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[239.16, 238.6, 240.49, 238.7] [238.57, 238.31, 238.2, 237.3]

UP(D)
UP(U)
UP(D)
2015-09-29 01:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[239.31,


UP(U)
UP(U)
2015-10-16 19:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[260.8, 258.79, 256.82] [258.51, 256.85, 256.51]
UP(U)
UP(D)
UP(U)
UP(U)
UP(U)
UP(U)
UP(U)
2015-10-17 02:00:00
['U', 'U', 'U', 'U', 'U', 'U']
[266.22, 265.59, 262.87, 261.97, 261.32, 260.6] [264, 262.92, 261.85, 261.09, 260.5, 259.79]
UP(U)


2015-10-17 05:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[266.02, 265.08, 267.55, 266.22, 265.59, 262.87] [265, 262.38, 265.1, 264, 262.92, 261.85]
DN(U)



UP(D)

UP(U)
2015-10-17 12:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[266.95, 264.75, 264.52, 263.89] [264.41, 263.28, 263, 261.84]


2015-10-17 14:00:00
['U', 'D', 'U', 'D', 'U', 'U']
[270.39, 267.55, 266.95, 264.75, 264.52, 263.89] [267.19, 266.05, 264.41, 263.28, 263, 261.84]

DN(U)


UP(U)
UP(D)
UP(U)
2015-10-17 21:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[272.09, 271.1, 271.79, 270.74, 268.67] [270.78, 269.62, 270.01, 268.2, 266.2]
UP(U)
UP(D)
2015-10-17 23:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[275, 278.51, 272.09, 271.1, 271.79, 270.74] [2


DN(D)
DN(D)
DN(U)
DN(D)
DN(D)
DN(D)
DN(D)
DN(D)
DN(D)
DN(U)
DN(D)



UP(D)
2015-11-01 10:00:00
['U', 'U', 'D', 'U', 'D', 'U']
[321.29, 317.8, 319.35, 317.4] [317.76, 314.48, 314.95, 310.92]




UP(U)
2015-11-01 15:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[322.51, 320.71, 316.93] [317.64, 316.43, 316.33]
UP(D)
UP(D)
UP(U)
UP(U)
2015-11-01 19:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[324.44, 320.42, 319.57] [320.45, 319, 318.39]
UP(U)


2015-11-01 22:00:00
['U', 'D', 'D', 'U', 'U', 'U']
[325.66, 323.6, 324.62, 324.44, 320.42, 319.57] [323.5, 322.21, 323.49, 320.45, 319, 318.39]
DN(U)
DN(D)
DN(D)
DN(D)
DN(D)

UP(U)
UP(U)
UP(U)
UP(U)
2015-11-02 08:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[334.51, 330.64, 328.27, 325.95, 321.19] [330.33, 326.22, 325.45, 320.69, 315.79]
UP(U)
UP(D)
2015-11-02 10:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[334.12, 335, 334.51, 330.64, 328.27, 325.95] [331.27, 330.46, 330.33, 326.22, 325.45, 320.69]


UP(D)

UP(U)
2015-11-02 15:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[333.7, 333.





DN(D)

UP(D)












2015-11-19 02:00:00
['U', 'D', 'D', 'U', 'U', 'D']
[335.63, 336.36, 337.74, 337.47, 334.35] [335.11, 334.95, 335.96, 333.49, 332.9]
DN(D)
DN(U)


UP(U)
UP(D)

DN(U)
DN(D)
DN(U)




DN(D)

UP(D)
UP(D)
UP(U)
2015-11-19 21:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[334.8, 334.51, 335.05, 333.9] [333.6, 333.6, 333.9, 332.99]
UP(U)
UP(D)

DN(D)
DN(D)
DN(U)


UP(U)
2015-11-20 06:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[330.15, 329.46, 329.73, 328.82] [328.79, 327.67, 328.11, 326.75]

DN(U)
DN(D)
DN(D)
DN(D)
DN(D)
DN(U)

UP(D)
UP(U)
2015-11-20 16:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[318.97, 318.23, 319.07, 318] [317, 316.81, 315.01, 312.7]
UP(U)
UP(D)
UP(U)

UP(U)
UP(U)
2015-11-20 22:00:00
['U', 'U', 'D', 'U', 'U', 'D']
[326.19, 323.4, 322.93, 322.86, 318.67] [323.83, 320.54, 320.42, 318.03, 314.82]

UP(D)
2015-11-21 00:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[324.73, 325.45, 326.19, 323.4] [323.07, 322, 323.83, 320.54]

DN(U)

DN(U)

UP(D)
UP(U)
UP(U)
UP(U)
UP(D)
UP(D)
201

2015-12-05 16:00:00
['U', 'D', 'U', 'U', 'D', 'U']
[373.5, 373.41, 374.45, 375.59] [372.28, 369.71, 371.21, 368.9]

DN(U)


UP(U)
2015-12-05 21:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[381.43, 378.44, 372.44] [374.42, 372.44, 370.64]
UP(U)



UP(D)
2015-12-06 02:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[381.89, 380.69, 380.78] [379.44, 379.74, 376.01]
UP(U)
UP(D)
UP(U)
2015-12-06 05:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[381.84, 380.93, 382.09, 381.89, 380.69, 380.78] [379.97, 379.55, 378.69, 379.44, 379.74, 376.01]

UP(D)
UP(U)

UP(U)

DN(U)


UP(U)
2015-12-06 15:00:00
['U', 'U', 'U', 'D', 'D', 'U']
[407.51, 402.12, 397.09] [399.2, 396.26, 393.94]
UP(D)


DN(U)
DN(D)

DN(U)

UP(D)
UP(U)
UP(U)
2015-12-07 02:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[399, 396.88, 394.67, 394.19] [394.61, 392.94, 392.25, 387.39]
UP(D)
UP(D)
UP(U)
UP(D)
UP(U)
2015-12-07 07:00:00
['U', 'U', 'U', 'U', 'D', 'U']
[403.82, 401.98, 402.82, 396.79] [400.05, 396.32, 395.73, 394.02]
UP(U)



UP(D)
UP(U)
2015-12-07 13:00:00
['U

...

DN(U)

UP(U)
2021-06-29 01:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[34950, 34531.5, 34588, 34518.5] [34300, 34220.5, 34156, 33879]


DN(D)
DN(U)









2021-06-29 14:00:00
['U', 'U', 'D', 'D', 'U', 'U']
[34969.5, 34615, 34840, 34997, 35175.5, 34477] [34533.5, 34211, 34198.5, 34510, 34380, 34019]

UP(U)
2021-06-29 16:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[35447, 34997, 34969.5, 34615] [34580, 34558.5, 34533.5, 34211]


UP(D)
UP(U)
2021-06-29 20:00:00
['U', 'U', 'U', 'D', 'U', 'D']
[35773.5, 35359.5, 35243.5] [35050.5, 34870, 34725]
UP(D)
UP(U)
UP(U)
2021-06-29 23:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[36383.5, 35975, 35740.5, 35773.5, 35359.5, 35243.5] [35731, 35350.5, 35500, 35050.5, 34870, 34725]

UP(D)
UP(U)
UP(U)
2021-06-30 03:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[36585, 36496, 36350] [36155.5, 36066.5, 35961]
UP(D)
UP(D)
2021-06-30 05:00:00
['U', 'D', 'U', 'U', 'U', 'D']
[36388, 36392, 36585, 36496, 36350] [36228, 36137, 36155.5, 36066.5, 35961]


2021-06-30 07:00:00
['U', 'D', 'U', '

DN(U)
DN(D)
DN(U)

UP(U)
2021-07-15 08:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[33045, 32905, 32832.5] [32745, 32788, 32713]
UP(D)
UP(U)

DN(D)
DN(D)
DN(D)
DN(U)
DN(D)
DN(D)
DN(D)
DN(U)
DN(D)




DN(D)
DN(D)
DN(D)
DN(U)

UP(D)
2021-07-16 06:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[31752, 31762, 31471] [31514, 31376, 31210]
UP(U)
UP(U)
UP(U)
2021-07-16 09:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[31904.5, 31854.5, 31792.5, 31752, 31762, 31471] [31510.5, 31628, 31623.5, 31514, 31376, 31210]

UP(D)
UP(U)
UP(D)
2021-07-16 13:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[32014.5, 32025, 31863.5] [31891, 31807, 31630]
UP(D)

DN(U)
DN(U)
DN(D)
DN(D)
DN(D)
DN(D)
DN(U)

UP(D)
UP(U)
2021-07-17 01:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[32180, 31935, 31974.5, 31810] [31804, 31755.5, 31709.5, 31070]
UP(U)
UP(D)
2021-07-17 03:00:00
['U', 'D', 'U', 'U', 'U', 'U']
[32114.5, 32240.5, 32180, 31935, 31974.5, 31810] [31934, 31807.5, 31804, 31755.5, 31709.5, 31070]

DN(U)
DN(D)
DN(D)
DN(D)
DN(D)
DN(D)
DN(U)
DN(D)
DN(U)
DN(D)


DN(D)

UP(U)
2021-08-01 07:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[41998.5, 41864, 41551, 41420] [41492, 41146.5, 41245.5, 41137]



DN(U)

UP(U)
2021-08-01 13:00:00
['U', 'U', 'D', 'U', 'D', 'D']
[42648, 41830, 41955.5, 41776.5] [41830, 41647, 41647, 41073]


DN(D)
DN(D)
DN(U)


UP(D)

DN(D)
DN(D)
DN(D)
DN(D)
DN(U)
DN(U)
DN(D)
DN(U)
DN(D)
DN(D)
DN(D)
DN(U)
DN(D)

UP(D)
2021-08-02 13:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[39924, 39848.5, 39740] [39710, 39430.5, 39205]
UP(D)
UP(D)
UP(U)
2021-08-02 16:00:00
['U', 'U', 'D', 'U', 'U', 'U']
[40297, 39798, 39832.5, 39924, 39848.5, 39740] [39630.5, 39567, 39626, 39710, 39430.5, 39205]








UP(D)
UP(U)
2021-08-03 02:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[39927, 39800.5, 39734, 39650] [39580.5, 39599, 39514.5, 39132.5]
UP(D)

DN(D)
DN(D)
DN(U)


UP(D)



DN(D)
DN(D)
DN(U)


UP(D)
UP(U)
2021-08-03 20:00:00
['U', 'U', 'D', 'U', 'U', 'D']
[38811, 38611.5, 38645.5, 38623.5, 38400] [38463, 38291.5, 38300, 38249.5, 38195.5]



DN(D)
DN(D)

DN(U)



U

DN(D)
DN(U)







DN(U)
DN(D)
DN(D)

UP(U)
2021-08-17 19:00:00
['U', 'U', 'U', 'D', 'D', 'D']
[46867, 46348.5, 46001] [46189.5, 45874.5, 45650]
UP(U)
UP(U)
2021-08-17 21:00:00
['U', 'D', 'U', 'U', 'U', 'D']
[47181, 46958, 46867, 46348.5, 46001] [46627.5, 46603, 46189.5, 45874.5, 45650]

DN(U)
DN(D)
DN(D)
DN(D)
DN(D)
DN(U)
DN(D)
DN(D)
DN(U)
DN(D)
DN(U)

UP(D)

UP(D)
UP(U)
UP(U)
2021-08-18 15:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[45398, 45248, 45048.5] [44985.5, 44850.5, 44600]
UP(U)







UP(D)
UP(U)
UP(U)
2021-08-19 02:00:00
['U', 'U', 'U', 'U', 'U', 'D']
[45990, 45824, 45490, 45400, 45100] [45594, 45202, 45141, 44989.5, 44649]
UP(D)

DN(U)
DN(D)
DN(D)


UP(D)
2021-08-19 10:00:00
['U', 'D', 'U', 'U', 'D', 'D']
[45060, 45164.5, 44895, 44772] [44480, 44683.5, 44501, 44400]

DN(U)
DN(D)
DN(D)


UP(D)
UP(U)
UP(D)
2021-08-19 19:00:00
['U', 'U', 'U', 'D', 'U', 'U']
[44865, 44975.5, 44650] [44606.5, 44367, 44211]
UP(D)



UP(U)
UP(U)
2021-08-20 01:00:00
['U', 'U', 'U', 'U', 'D', 'D']
[45807

TypeError: string indices must be integers